In [1]:
import requests
import pickle
import datetime

# This script interacts with the Santa Monica Open Data API to extract daily parking counts

The idea is that some version of this script can be run daily to generated the needed parking lot features for upcoming markets

In [11]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.smgov.net", 'gh0PGNOdsbpE9OEaUFU2DW2Y2')

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.smgov.net,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("ng8m-khuz", limit=50000)

# Convert to pandas DataFrame

# Store 50,000 most recent parking lot counts into pandas DF

Here 50,000 is chosen to ensure counts from at least the least 24 hours are included in the DF

In [30]:
results_df = pd.DataFrame.from_records(results)
results_df['date_time']=pd.to_datetime(results_df['date_time'])
results_df['date']=results_df['date_time'].apply(lambda x: x.date())

# Filter data by lot number

In [43]:
results_df.sort_values(by='date_time')
lots_to_keep=['Structure 5','Structure 1','Lot 4 South','Structure 2','Structure 3','Structure 6','Structure 8','Structure 9','Structure 7']
results_df=results_df[results_df['lot_name'].apply(lambda x: x in lots_to_keep)]
results_df['available_spaces']=results_df['available_spaces'].apply(lambda x: int(x))

# Calculate average occupany during peak hours

In [54]:
#look at the parking in the peak hours
peak_parking_av=results_df[results_df['date_time'].apply(lambda x: True if x.hour>=9 and x.hour<=20 else False)].groupby(by=['date']).mean()['available_spaces'].reset_index()

# Normalize by overnight counts

In [58]:
#look at the parking in the morning hours
morning_parking_av=results_df[results_df['date_time'].apply(lambda x: True if x.hour>=2 and x.hour<9 else False)].groupby(by=['date']).mean()['available_spaces'].reset_index()
morning_parking_av=morning_parking_av.rename(columns={"date": "date", "available_spaces": "TAKEN"})

In [62]:
joined_parking_df=peak_parking_av.set_index('date').join(morning_parking_av.set_index('date'),lsuffix='date',rsuffix='date').reset_index().sort_values(by='date')                                                                                                                              

In [63]:
joined_parking_df['NET PEAK SPOTS']=joined_parking_df['available_spaces']-joined_parking_df['TAKEN']
joined_parking_df['NET PEAK SPOTS']=joined_parking_df['NET PEAK SPOTS']+245
joined_parking_df=joined_parking_df[['date','NET PEAK SPOTS']].sort_values(by=['date'])

# Save output

In [66]:
os.chdir('C:\\Users\\teek2\\Documents\\Farmers Market Forecast\\Feature_Data_Files')
joined_parking_df[['date','NET PEAK SPOTS']].to_pickle('scraped_parking_data.pkl')